# Building a blockchain in python

Inspired from [Python Tutorial: Build A Blockchain In < 60 Lines of Code](https://medium.com/coinmonks/python-tutorial-build-a-blockchain-713c706f6531) and [Learn Blockchains by Building One](https://hackernoon.com/learn-blockchains-by-building-one-117428612f46).

In [16]:
import datetime
import hashlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## The SHA-256 function

In [17]:
hashlib.sha256(b"Is DeFI a scam").hexdigest()

'60e37b5ad2f6a9cd8fe8a0bfa929e96181488dd6fd48748c04657d44f17479bb'

## The block class

In [5]:
class Block:
    hash = None # Hash of the block info
    index = 0 # Index of the block inside the chain
    timestamp = datetime.datetime.now() # Time of creation of the block
    nonce = 0 # Solution to the cryptopuzzle
    transactions = [] # Transaction data
    mined = False # Boolean set to True whenever the problem has been solved
    previous_hash = 0x0 # Hash of the previous block
    

    def __init__(self, transactions):
        self.transactions = transactions
        self.timestamp = datetime.datetime.now()

    def hash(self):# Compute the hash of the blockdata
        h = hashlib.sha256()
        h.update(
        str(self.nonce).encode('utf-8') +
        str(self.transactions).encode('utf-8') +
        str(self.previous_hash).encode('utf-8') 
        )
        return h.hexdigest()
    
    # Add a new transaction to a block
    def new_transaction(self, sender, recipient, amount, fee):
        transaction = {
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
            'fee' : fee
        }
        self.transactions.append(transaction)
    
    # Print the block info
    def __str__(self):
        return "Block Height: " + str(self.index) + \
    "\nBlock Hash: " + str(self.hash()) + \
    "\nTime:" + str(self.timestamp) + \
    "\nBlock data: " + str(self.transactions) + \
    "\nMined: " + str(self.mined) + \
    "\nPrevious block hash: " + str(self.previous_hash) +"\n--------------"
    

    


In [6]:
B1 = Block([])
B1.new_transaction("Coinbase", "Satoshi", 100, 0)
B1.new_transaction("Satoshi", "Pierre-O", 5, 2)
print(B1)


Block Height: 0
Block Hash: 99f27c54043f4464a1f7a2ef0960a68c4ad07f3f0b863c57023b5a19efdf9a7c
Time:2023-10-26 13:54:38.942478
Block data: [{'sender': 'Coinbase', 'recipient': 'Satoshi', 'amount': 100, 'fee': 0}, {'sender': 'Satoshi', 'recipient': 'Pierre-O', 'amount': 5, 'fee': 2}]
Mined: False
Previous block hash: 0
--------------


## The blockchain class

In [7]:
class Blockchain:
    diff = 0
    maxNonce = 2**32
    target = 2 ** (256-diff)
    reward = 50
    miner = "Miner"

    def __init__(self, genesis_block):
        self.chain = [genesis_block]
        self.pending_transactions = []
        
    def new_transaction(self, sender, recipient, amount, fee):
        transaction = {
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
            'fee' : fee
        }
        self.pending_transactions.append(transaction) 
    
    def add_block(self):
#         blockchain.pending_transactions
        current_block = blockchain.chain[-1]
        new_block = Block(blockchain.pending_transactions)
        new_block.index = current_block.index + 1
        new_block.previous_hash = current_block.hash()
        blockchain.chain.append(new_block)
        blockchain.pending_transactions = []
    
    def adjust_difficulty(self, new_diff):
        self.diff = new_diff
        self.target = 2 ** (256-new_diff)
    
    def halve_reward(self):
        self.reward = self.reward / 2
    
    def mine(self):
        block = self.chain[-1]
        target = self.target

        if block.transactions:
            fee = pd.DataFrame.from_records(block.transactions).fee.sum()
        else:
            fee = 0

        block.new_transaction("Coinbase", self.miner, self.reward + fee, 0)
        while int(block.hash(), 16) > target:
            block.nonce = int(np.random.uniform(low = 0, high = 2**32 + 1))
        block.mined = True
        block.timestamp = datetime.datetime.now() 
        



In [13]:
genesis_block = Block([])
blockchain = Blockchain(genesis_block)
print(blockchain.chain[-1])
blockchain.adjust_difficulty(20)
blockchain.halve_reward()
blockchain.mine()
print(blockchain.chain[-1])

Block Height: 0
Block Hash: 32369d15916932bd1ade51c0cae9f32f5ded2cccd29d81d7eec27066c813e39d
Time:2023-10-26 13:56:52.734213
Block data: []
Mined: False
Previous block hash: 0
--------------
Block Height: 0
Block Hash: 00000ced170abe0f4c8f141457f4ca55dc8de036736145353d5443d12b16d8df
Time:2023-10-26 13:56:56.462050
Block data: [{'sender': 'Coinbase', 'recipient': 'Miner', 'amount': 25.0, 'fee': 0}]
Mined: True
Previous block hash: 0
--------------


In [14]:
blockchain.new_transaction("miner", "Pierre-O", 5, 0.1)
blockchain.new_transaction("miner", "Satoshi", 10, 0.2)
# print(blockchain.pending_transactions)
blockchain.add_block()
for block in blockchain.chain: 
    print(block)

Block Height: 0
Block Hash: 00000ced170abe0f4c8f141457f4ca55dc8de036736145353d5443d12b16d8df
Time:2023-10-26 13:56:56.462050
Block data: [{'sender': 'Coinbase', 'recipient': 'Miner', 'amount': 25.0, 'fee': 0}]
Mined: True
Previous block hash: 0
--------------
Block Height: 1
Block Hash: a43ec14b95d2f81b65e45aad7033e39f7d580c64e0471cbc538890541a2a1a64
Time:2023-10-26 13:57:07.463098
Block data: [{'sender': 'miner', 'recipient': 'Pierre-O', 'amount': 5, 'fee': 0.1}, {'sender': 'miner', 'recipient': 'Satoshi', 'amount': 10, 'fee': 0.2}]
Mined: False
Previous block hash: 00000ced170abe0f4c8f141457f4ca55dc8de036736145353d5443d12b16d8df
--------------


In [15]:
blockchain.mine()
for block in blockchain.chain: 
    print(block)

Block Height: 0
Block Hash: 00000ced170abe0f4c8f141457f4ca55dc8de036736145353d5443d12b16d8df
Time:2023-10-26 13:56:56.462050
Block data: [{'sender': 'Coinbase', 'recipient': 'Miner', 'amount': 25.0, 'fee': 0}]
Mined: True
Previous block hash: 0
--------------
Block Height: 1
Block Hash: 000005942a54b5e79056fe736dd8fa099a6249c831b55545327bc43b281268e9
Time:2023-10-26 13:57:26.212125
Block data: [{'sender': 'miner', 'recipient': 'Pierre-O', 'amount': 5, 'fee': 0.1}, {'sender': 'miner', 'recipient': 'Satoshi', 'amount': 10, 'fee': 0.2}, {'sender': 'Coinbase', 'recipient': 'Miner', 'amount': 25.3, 'fee': 0}]
Mined: True
Previous block hash: 00000ced170abe0f4c8f141457f4ca55dc8de036736145353d5443d12b16d8df
--------------
